In [22]:
from __future__ import annotations

import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn

from scipy.integrate import solve_ivp
from scipy.integrate import odeint

import scipy

import sys
import gymnasium as gym
import time
import signal

# 定义环境的相关设置
balance_time = 240
h_in = 1 / 100

# 注册和创建自定义环境
CartPoleSwingUp = gym.register(
    id = 'CartPoleSwingUp',
    entry_point = 'myCartpoleF_SwingUp:CartPoleSwingUp',  # 根据你的环境路径修改
    reward_threshold = -40 * 0.95,
    max_episode_steps = int(balance_time / h_in),
)
env = gym.make('CartPoleSwingUp', render_mode='human')
print(gym.spec('CartPoleSwingUp'))

EnvSpec(id='CartPoleSwingUp', entry_point='myCartpoleF_SwingUp:CartPoleSwingUp', reward_threshold=-38.0, nondeterministic=False, max_episode_steps=24000, order_enforce=True, disable_env_checker=False, kwargs={}, namespace=None, name='CartPoleSwingUp', version=None, additional_wrappers=(), vector_entry_point=None)


创建 NormalActionNoise 实例，用于在智能体的动作中加入噪声。具体的参数含义如下：

mean = np.zeros(env.action_space.shape): 这里的 mean 是噪声的均值，np.zeros(env.action_space.shape) 表示均值为零，且它的维度与环境的动作空间 env.action_space.shape 相同。env.action_space.shape 表示动作空间的维度。

sigma = 0.1 * np.ones(env.action_space.shape): 这里的 sigma 是噪声的标准差，0.1 * np.ones(env.action_space.shape) 表示标准差是 0.1，并且与动作空间的维度相同。这样每个维度的噪声标准差都是 0.1。

通过这种方式，动作中会加入一定的噪声，从而使得训练过程更加稳定，防止模型陷入局部最优解，增强探索性。

创建 DDPG 模型：

'MlpPolicy': 选择了多层感知器（MLP）作为策略网络的结构。该网络用于从状态中输出动作。

env: 环境对象，智能体将在该环境中进行训练。

policy_kwargs = dict(net_arch=[400, 300]): 这个参数定义了策略网络的结构，net_arch=[400, 300] 表示策略网络有两层隐藏层，分别有 400 个神经元和 300 个神经元。policy_kwargs 是传递给策略网络构造函数的附加参数。

verbose = 1: 这个参数设置了训练时的输出级别，verbose=1 表示打印训练过程中的信息。

tensorboard_log = "./ddpg_cartpole/": 这个参数指定了 TensorBoard 日志的存储路径，用于后期可视化训练过程。

action_noise=action_noise: 将前面创建的噪声对象 action_noise 传递给 DDPG 模型，用于在选择动作时加入噪声。

In [23]:
# 步骤三：使用 DDPG 算法训练代理
from stable_baselines3 import DDPG
from stable_baselines3.common.vec_env import DummyVecEnv

# 将环境包装为向量环境
env = DummyVecEnv([lambda: gym.make('CartPoleSwingUp', render_mode='human')])

# 初始化 DDPG 代理
from stable_baselines3.common.noise import NormalActionNoise
# 加入噪声
action_noise = NormalActionNoise(mean = np.zeros(env.action_space.shape), sigma = 0.2 * np.ones(env.action_space.shape))
model = DDPG('MlpPolicy', env, policy_kwargs = dict(net_arch=[256, 128]), action_noise = action_noise, verbose = 1)

model.learn(total_timesteps = 100000)

model.save("ddpg_cartpole_model")
# 评估训练后的模型
from stable_baselines3.common.evaluation import evaluate_policy

model = DDPG.load("ddpg_cartpole_model")
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes = 10)
print(f"Mean reward: {mean_reward}, Std reward: {std_reward}")

# 测试模型
obs = env.reset()
done = False
while not done:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = env.step(action)
    env.render()  # 可视化环境

env.close()


Using cpu device
---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 63       |
|    time_elapsed    | 6        |
|    total_timesteps | 428      |
| train/             |          |
|    actor_loss      | 249      |
|    critic_loss     | 1.08e+06 |
|    learning_rate   | 0.001    |
|    n_updates       | 327      |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 60       |
|    time_elapsed    | 9        |
|    total_timesteps | 565      |
| train/             |          |
|    actor_loss      | 369      |
|    critic_loss     | 1.07e+06 |
|    learning_rate   | 0.001    |
|    n_updates       | 464      |
---------------------------------
---------------------------------
| time/              |          |
|    episodes        | 12       |
|    fps             | 58       |
|    time_elapsed    | 12      

Traceback (most recent call last):
  File "C:\Users\ZPD\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\ZPD\AppData\Local\Temp\ipykernel_56828\740549276.py", line 14, in <module>
    model.learn(total_timesteps = 100000)
  File "d:\Study\anaconda\envs\new\lib\site-packages\stable_baselines3\ddpg\ddpg.py", line 123, in learn
    return super().learn(
  File "d:\Study\anaconda\envs\new\lib\site-packages\stable_baselines3\td3\td3.py", line 222, in learn
    return super().learn(
  File "d:\Study\anaconda\envs\new\lib\site-packages\stable_baselines3\common\off_policy_algorithm.py", line 328, in learn
    rollout = self.collect_rollouts(
  File "d:\Study\anaconda\envs\new\lib\site-packages\stable_baselines3\common\off_policy_algorithm.py", line 560, in collect_rollouts
    new_obs, rewards, dones, infos = env.step(actions)
  File "d:\Study\anaconda\envs\new\lib\site